<a href="https://colab.research.google.com/github/vishallmaurya/sentiment_analyzer_backend/blob/main/sentiment_analysis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 30.3 MB/s eta 0:00:00


In [84]:
import numpy as np
import pandas as pd
import re
import emoji
import nltk
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import PassiveAggressiveClassifier

In [25]:
data = pd.read_csv('/content/sentiment_data_2_label_annotated.csv')
data1 = pd.read_csv('/content/sentiment_data_3csv_annotated.csv')
data2 = pd.read_csv('/content/sentiment_data_label_annotated.csv')
# data3 = pd.read_csv('/content/synthetic_sentiment_data.csv')

In [26]:
data = pd.concat([data, data1, data2])

In [27]:
data.head(1)

,Unnamed: 0,Date,Tweet,class,gpt_label
0,0,2022-12-31 23:52:40+00:00,Waited longer for one checked bag than the act...,2,0


In [28]:
data.drop(['Unnamed: 0', 'Date', 'class'], inplace=True, axis=1)

In [29]:
nltk.download('stopwords')

all_stopwords = stopwords.words('english')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
data['gpt_label'].value_counts()

,count
gpt_label,
1,528
0,231
2,103


In [31]:
def extract_emojis(text):
    return ''.join(c for c in text if c in emoji.EMOJI_DATA)

def cleaning_tweets(tweet):
    extracted_emojis = extract_emojis(tweet)
    tweet = re.sub(r'\\p{So}', '', tweet)
    tweet = re.sub('[^a-zA-Z]', ' ', tweet)
    tweet = tweet.lower()
    tweet = tweet.split()

    tweet = [ps.stem(word) for word in tweet if word not in all_stopwords]
    tweet = ' '.join(tweet)

    return tweet, extracted_emojis


In [32]:
cleaned_tweets = []
emoji_list = []

for i in range(data.shape[0]):
  clean_text, extracted_emojis = cleaning_tweets(str(data.iloc[i]["Tweet"]))
  cleaned_tweets.append(clean_text)
  emoji_list.append(extracted_emojis)

data["clean_tweet"] = cleaned_tweets
data["extracted_emojis"] = emoji_list

In [33]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def emoji_to_text(emoji_string):
    return ' '.join(emoji.demojize(e) for e in emoji_string) if emoji_string else ''

data["emoji_text"] = data["extracted_emojis"].apply(emoji_to_text)

text_embeddings = model.encode(data["clean_tweet"].tolist(), convert_to_numpy=True)
data["text_embeddings"] = list(text_embeddings)

embedding_dim = model.get_sentence_embedding_dimension()

emoji_embeddings = [
    model.encode(emoji_text, convert_to_numpy=True) if emoji_text.strip() else np.zeros(embedding_dim)
    for emoji_text in data["emoji_text"]
]

data["emoji_embeddings"] = list(emoji_embeddings)

In [34]:
data.head(1)

,Tweet,gpt_label,clean_tweet,extracted_emojis,emoji_text,text_embeddings,emoji_embeddings
0,Waited longer for one checked bag than the act...,0,wait longer one check bag actual flight thank ...,,,"[0.022148889, 0.04465435, -0.0048997323, 0.046...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [66]:
from imblearn.over_sampling import SMOTE
smote_balance = SMOTE()

X_text = np.array(data["text_embeddings"].tolist())
X_emoji = np.array(data["emoji_embeddings"].tolist())


X_final = np.hstack((X_text, X_emoji))


y = data["gpt_label"].values

smote = SMOTE()
X_SMOTE, Y_SMOTE = smote.fit_resample(X_final, y)

In [75]:
X_SMOTE = X_final
Y_SMOTE = data["gpt_label"].values

In [76]:

X_train1, X_test1, y_train1, y_test1 = train_test_split(X_SMOTE, Y_SMOTE, test_size=0.2, random_state=42)


In [86]:
base_weights = {0: 1, 1: 1, 2: 1}

def adjust_class_weights(y_real_time, base_weights):
    class_counts = Counter(y_real_time)
    total = sum(class_counts.values())

    new_weights = {cls: base_weights.get(cls, 1) * (total / (class_counts[cls] + 1)) for cls in [0, 1, 2]}

    return new_weights

In [87]:
for X_batch, y_batch in X_final, y:  # Replace this with your actual data stream
    updated_weights = adjust_class_weights(y_batch, base_weights)  # Update class weights dynamically

    model = SGDClassifier(loss="hinge", class_weight=updated_weights)  # Apply new weights

    model.partial_fit(X_batch, y_batch, classes=[0, 1, 2])

ValueError: too many values to unpack (expected 2)

In [60]:
# rf_model = ExtraTreesClassifier(n_estimators=161,warm_start=True, random_state=42)
# rf_model.fit(X_train1, y_train1)

# # Evaluate accuracy
# y_pred1 = rf_model.predict(X_test1)
# print(f"Extra Forest Accuracy: {accuracy_score(y_test1, y_pred1):.4f}")

In [77]:
incremental_clf = SGDClassifier(loss="perceptron", warm_start=True, class_weight="balanced")
incremental_clf.fit(X_train1, y_train1)

SGDClassifier(class_weight='balanced', loss='perceptron', warm_start=True)

In [78]:
y_pred1 = incremental_clf.predict(X_test1)
print(f"SGD Accuracy: {accuracy_score(y_test1, y_pred1): .4f}" )

SGD Accuracy:  0.7514


In [79]:
pa_classifier = PassiveAggressiveClassifier( max_iter=1000,  # Increase iterations
    C=0.5,          # Adjust regularization strength (try 0.1, 0.5, 1.0)
    loss="hinge",   # Test 'squared_hinge' for better margin separation
    tol=1e-4,       # Reduce tolerance for better convergence
    class_weight="balanced",
    warm_start=True)
pa_classifier.fit(X_train1, y_train1)

PassiveAggressiveClassifier(C=0.5, class_weight='balanced', tol=0.0001,
                            warm_start=True)

In [80]:
y_pred1 = pa_classifier.predict(X_test1)
print(f"Passive Aggressive Accuracy: {accuracy_score(y_test1, y_pred1): .4f}" )

Passive Aggressive Accuracy:  0.6936


In [82]:
batch_size = 50
for i in range(0, len(X_train1), batch_size):
    X_batch = X_train1[i:i + batch_size]
    y_batch = y_train1[i:i + batch_size]

    if i == 0:
        pa_classifier.partial_fit(X_batch, y_batch, classes=np.unique(y_train1))
    else:
        pa_classifier.partial_fit(X_batch, y_batch)


In [83]:
y_pred = pa_classifier.predict(X_test1)
print(f"PassiveAggressiveClassifier Accuracy: {accuracy_score(y_test1, y_pred):.4f}")

PassiveAggressiveClassifier Accuracy: 0.6936
